In [5]:
from cabi_functions import *
import pandas as pd

In [2]:
stations = load_stations()
trips = return_trip_datatable()
trips.dropna(inplace=True)

In [12]:
stations

,eightd_station_services,rental_uris,legacy_id,lon,external_id,electric_bike_surcharge_waiver,rental_methods,lat,name,station_id,station_type,capacity,region_id,has_kiosk,short_name,eightd_has_key_dispenser,geometry
0,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,1,-77.053230,082469cc-1f3f-11e7-bf6b-3863bb334450,False,"[CREDITCARD, KEY]",38.858971,Eads St & 15th St S,1,classic,15,41,True,31000,False,POINT (38.85897 -77.05323)
1,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,3,-77.049232,08246c35-1f3f-11e7-bf6b-3863bb334450,False,"[CREDITCARD, KEY]",38.856425,Crystal Dr & 20th St S,3,classic,17,41,True,31002,False,POINT (38.85643 -77.04923)
2,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,4,-77.049417,08246cd5-1f3f-11e7-bf6b-3863bb334450,False,"[CREDITCARD, KEY]",38.861056,Crystal Dr & 15th St S,4,classic,16,41,True,31003,False,POINT (38.86106 -77.04942)
3,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,5,-77.059490,08246d68-1f3f-11e7-bf6b-3863bb334450,False,"[CREDITCARD, KEY]",38.857866,Aurora Hills Cmty Ctr / 18th St & S Hayes St,5,classic,12,41,True,31004,False,POINT (38.85787 -77.05949)
4,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,6,-77.059936,08246df5-1f3f-11e7-bf6b-3863bb334450,False,"[CREDITCARD, KEY]",38.862303,Pentagon City Metro / 12th St & S Hayes St,6,classic,18,41,True,31005,False,POINT (38.86230 -77.05994)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,696,-77.116339,1cc6ee60-b82f-4264-a490-6c3e5e656ccd,False,"[CREDITCARD, KEY]",38.857049,8th Rd & S Frederick St,696,classic,12,41,True,31951,False,POINT (38.85705 -77.11634)
649,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,697,-76.996023,0627dc3c-5c62-4ab1-a6f0-3635c6f80d23,False,"[CREDITCARD, KEY]",38.922404,MBT & Bryant St NE,697,classic,19,42,True,31531,False,POINT (38.92240 -76.99602)
650,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,698,-77.105564,5e229707-ae42-42f7-b31d-1a65af253027,False,"[CREDITCARD, KEY]",38.818509,N Howard St & N Imboden St,698,classic,15,40,True,31952,False,POINT (38.81851 -77.10556)
651,[],{'android': 'https://dc.lft.to/lastmile_qr_sca...,701,-77.222808,c271d265-76c8-49f9-9b3c-b9490fb97ac0,False,"[CREDITCARD, KEY]",38.878601,Hartland Rd & Harte Pl,701,classic,11,104,True,32255,False,POINT (38.87860 -77.22281)


Create lists of different variables for the trips table, then assign them to columns in the table

In [3]:
trips = trips.assign(
    time = trips.ended_at-trips.started_at,
    pair = pair_stations(trips.start_station_id,trips.end_station_id)
    )


In [4]:
trips = trips[trips.start_station_id!=trips.end_station_id].copy()

In [5]:
unidir_routes = trips.loc[:,['pair']].value_counts()
unidir_routes = pd.DataFrame(unidir_routes).reset_index().rename({0:'popularity'},axis=1)


In [6]:
st = [str(x)[:5] for x in unidir_routes.pair]
end = [str(x)[-5:] for x in unidir_routes.pair]
unidir_routes = pd.DataFrame(unidir_routes)
unidir_routes = unidir_routes.assign(startid=st,endid=end)

In [7]:
geom = [make_trip_geometry(*coords) for coords in zip(unidir_routes.startid,unidir_routes.endid)]


In [8]:
unidir_routes = unidir_routes.assign(geo=geom).dropna()

Add time to each trip

In [16]:
unidir_gdf = gpd.GeoDataFrame(unidir_routes,geometry='geo',crs=4326)
unidir_routes = unidir_routes.assign(length=unidir_gdf.to_crs(epsg=26985).length)

In [17]:
trips = trips.join(unidir_routes.loc[:,['pair','length']].set_index('pair'),on='pair',rsuffix='r')
trips.dropna(inplace=True)

trips = trips[trips.time.dt.total_seconds()>0].copy()
trips = trips.assign(avg_speed=trips.length/trips.time.dt.total_seconds())
avg_speed = trips.dropna().groupby('pair').mean().sort_values('avg_speed',ascending=False).avg_speed
unidir_routes = unidir_routes.set_index('pair').join(avg_speed,rsuffix='r')
gpd.GeoDataFrame(unidir_routes.reset_index(drop=True),geometry='geo').to_file('outputs/uni_speed.gpkg',driver='GPKG')

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['pair'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"